In [ ]:
%pip install onnx

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision.datasets as datasets

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Device : {device}") 

Device : cuda:0


In [2]:
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data',train=True,download=True,transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data',train=False,download=True,transform=transforms_cifar)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#To determine Custom Normalization Metrics
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset
from tqdm.notebook import tqdm
from time import time

N_CHANNELS = 1

dataset = datasets.MNIST("data", download=True,
                 train=True, transform=transforms.ToTensor())
full_loader = torch.utils.data.DataLoader(dataset, shuffle=False, num_workers=os.cpu_count())

before = time()
mean = torch.zeros(1)
std = torch.zeros(1)
print('==> Computing mean and std..')
for inputs, _labels in tqdm(full_loader):
    for i in range(N_CHANNELS):
        mean[i] += inputs[:,i,:,:].mean()
        std[i] += inputs[:,i,:,:].std()
mean.div_(len(dataset))
std.div_(len(dataset))
print(mean, std)

print("time elapsed: ", time()-before)

In [3]:
train_loaders = torch.utils.data.DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=2)
test_loaders = torch.utils.data.DataLoader(test_dataset,batch_size=128,shuffle=False,num_workers=2)

In [6]:
class DeepNN(nn.Module):
    def __init__(self, num_classes = 10):
        super(DeepNN,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(128,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(2048,512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512,num_classes)
        )

    def forward(self,X):
        X = self.features(X)  
        X = torch.flatten(X,1)
        X = self.classifier(X)
        return X

In [5]:
class LightNN(nn.Module):
    def __init__(self,num_classes = 10):
      super(LightNN,self).__init__()
      self.features = nn.Sequential(
         nn.Conv2d(3,16,kernel_size=3,padding=1),
         nn.ReLU(),
         nn.MaxPool2d(kernel_size=2,stride=2),
         nn.Conv2d(16,16,kernel_size=3,padding=1),
         nn.ReLU(),
         nn.MaxPool2d(kernel_size=2,stride=2)
      )

      self.classifier = nn.Sequential(
        nn.Linear(1024,256),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(256,num_classes)

      )

    def forward(self,X):
       X = self.features(X)
       X = torch.flatten(X,1)
       X = self.classifier(X)
       return X


In [6]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion  = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
 
    model.train()
    for epoch in range(epochs):
        runing_loss = 0.0
        for input, label in train_loader:
            input , label = input.to(device), label.to(device)

            optimizer.zero_grad()
            output = model(input)
            loss = criterion(output,label)
            loss.backward()
            optimizer.step()

            runing_loss += loss.item()

        print(f"Epoch {epoch +1}/ {epochs} , Loss: {runing_loss / len(train_loader)}")


def test(model,test_loader,device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [7]:
torch.manual_seed(42)
nn_deep = DeepNN(num_classes=10).to(device)
train(nn_deep,train_loaders,50,5e-5,device)
test_accuracy_deep = test(nn_deep,test_loaders,device)

torch.manual_seed(42)
nn_light = LightNN(num_classes=10).to(device)

Epoch 1/ 50 , Loss: 1.804095080136643
Epoch 2/ 50 , Loss: 1.4708543390874058
Epoch 3/ 50 , Loss: 1.3569431280540993
Epoch 4/ 50 , Loss: 1.271607019102482
Epoch 5/ 50 , Loss: 1.2065557808522374
Epoch 6/ 50 , Loss: 1.1477777526506683
Epoch 7/ 50 , Loss: 1.0926402209664854
Epoch 8/ 50 , Loss: 1.041032755618815
Epoch 9/ 50 , Loss: 0.9953873357199647
Epoch 10/ 50 , Loss: 0.9544498040852949
Epoch 11/ 50 , Loss: 0.9153146877922975
Epoch 12/ 50 , Loss: 0.8775202926162564
Epoch 13/ 50 , Loss: 0.8434965726359726
Epoch 14/ 50 , Loss: 0.8122409428172099
Epoch 15/ 50 , Loss: 0.7816535925011501
Epoch 16/ 50 , Loss: 0.7487331043423899
Epoch 17/ 50 , Loss: 0.7224821343141443
Epoch 18/ 50 , Loss: 0.6871171735436715
Epoch 19/ 50 , Loss: 0.663576886781951
Epoch 20/ 50 , Loss: 0.6347559724012604
Epoch 21/ 50 , Loss: 0.6047375401877382
Epoch 22/ 50 , Loss: 0.5815140062280933
Epoch 23/ 50 , Loss: 0.5551274838045125
Epoch 24/ 50 , Loss: 0.5317892457365685
Epoch 25/ 50 , Loss: 0.5087172233540079
Epoch 26/ 50 

In [27]:
torch.save(nn_deep.state_dict(), "final_BLSTM_model.pth")
print(f"Final model saved!")

Final model saved!


In [8]:
torch.manual_seed(42)
new_nn_light = LightNN(num_classes=10).to(device) 
print('Norm of 1st Layer of nn_light', torch.norm(nn_light.features[0].weight).item())
print("Norm of 1st Layer of new_nn_light:", torch.norm(new_nn_light.features[0].weight).item())

Norm of 1st Layer of nn_light 2.327361822128296
Norm of 1st Layer of new_nn_light: 2.327361822128296


In [9]:
total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 1,186,986
LightNN parameters: 267,738


In [10]:
train(nn_light, train_loaders, epochs=50, learning_rate=5e-5, device=device)
test_accuracy_light_ce = test(nn_light, test_loaders, device)

Epoch 1/ 50 , Loss: 1.979577873064124
Epoch 2/ 50 , Loss: 1.6811188066097171
Epoch 3/ 50 , Loss: 1.561907938679161
Epoch 4/ 50 , Loss: 1.4894302720608918
Epoch 5/ 50 , Loss: 1.433642453854651
Epoch 6/ 50 , Loss: 1.3907219924585288
Epoch 7/ 50 , Loss: 1.352692410464177
Epoch 8/ 50 , Loss: 1.3204457982421836
Epoch 9/ 50 , Loss: 1.2930744776640402
Epoch 10/ 50 , Loss: 1.2680785887686492
Epoch 11/ 50 , Loss: 1.2470414239122434
Epoch 12/ 50 , Loss: 1.2286432600387223
Epoch 13/ 50 , Loss: 1.2091225993907666
Epoch 14/ 50 , Loss: 1.1921184093446073
Epoch 15/ 50 , Loss: 1.1772517804294595
Epoch 16/ 50 , Loss: 1.161402896389632
Epoch 17/ 50 , Loss: 1.1468045964570301
Epoch 18/ 50 , Loss: 1.1341804839156169
Epoch 19/ 50 , Loss: 1.1203850059558058
Epoch 20/ 50 , Loss: 1.1097121276818882
Epoch 21/ 50 , Loss: 1.0942824571333882
Epoch 22/ 50 , Loss: 1.0808643308441963
Epoch 23/ 50 , Loss: 1.071204699671177
Epoch 24/ 50 , Loss: 1.057097227067289
Epoch 25/ 50 , Loss: 1.0488391821951513
Epoch 26/ 50 , L

In [28]:
torch.save(nn_light.state_dict(), "NNLight.pth")
print("Final model saved!")

Final model saved!


In [11]:
class ModifiedDeepNN(nn.Module):
    def __init__(self,num_classes = 10):
        super(ModifiedDeepNN,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,128,padding=1,kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(128,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048,512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512,num_classes)
        )
    
    def forward(self, X):
        X = self.features(X)
        flattened_conv_output = torch.flatten(X,1)
        X = self.classifier(flattened_conv_output)
        flattened_conv_output_after_pooling = torch.nn.functional.avg_pool1d(flattened_conv_output,2)
        return X, flattened_conv_output_after_pooling


class ModifiedLightNN(nn.Module):
    def __init__(self,num_classes = 10):
        super(ModifiedLightNN,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,16,padding=1,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(16,16,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024,256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256,num_classes)
        )
    

    def forward(self,X):
        X = self.features(X)
        flattened_conv_output = torch.flatten(X,1)
        X = self.classifier(flattened_conv_output)
        return X, flattened_conv_output


modified_nn_deep = ModifiedDeepNN(num_classes=10).to(device)
modified_nn_deep.load_state_dict(nn_deep.state_dict())

print("Norm of 1st layer for deep_nn:", torch.norm(nn_deep.features[0].weight).item())
print("Norm of 1st layer for modified_deep_nn:", torch.norm(modified_nn_deep.features[0].weight).item())

torch.manual_seed(42)
modified_nn_light = ModifiedLightNN(num_classes=10).to(device)
print("Norm of 1st layer:", torch.norm(modified_nn_light.features[0].weight).item())


Norm of 1st layer for deep_nn: 7.2851409912109375
Norm of 1st layer for modified_deep_nn: 7.2851409912109375
Norm of 1st layer: 2.327361822128296


In [14]:
#for Development Purpose - Giving a seed to check for Dimension 
sample_input = torch.randn(128,3,32,32).to(device)
logits , hidden_representation = modified_nn_light(sample_input)
print("Student logits Shape:" , logits.shape) #batch_size x totalclass
print("Student Hidden Representation Shape:", hidden_representation.shape) # batchsize X hiddenrep
logits , hidden_representation = modified_nn_deep(sample_input)
print("Teacher logits Shape:" , logits.shape) #batch_size x totalclass
print("Teacher Hidden Representation Shape:", hidden_representation.shape) # batchsize X hiddenrep

Student logits Shape: torch.Size([128, 10])
Student Hidden Representation Shape: torch.Size([128, 1024])
Teacher logits Shape: torch.Size([128, 10])
Teacher Hidden Representation Shape: torch.Size([128, 1024])


MaxPolling Cosine Changes Training

In [16]:
def train_cosine_loss(
        teacher,
        student,
        train_loaders,
        epochs,
        learning_rate,
        hidden_rep_loss_weight,
        ce_loss_weight,
        device
):
    ce_loss = nn.CrossEntropyLoss()
    cosine_loss = nn.CosineEmbeddingLoss()
    optimizer = optim.Adam(student.parameters(),lr = learning_rate)

    teacher.to(device)
    student.to(device)
    teacher.eval()
    student.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs , labels in train_loaders:
            inputs , labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with torch.no_grad():
                _, teacher_hidden_representation = teacher(inputs)
            
            studen_logits, student_hidden_representation = student(inputs)

            hidden_rep_loss = cosine_loss(student_hidden_representation,teacher_hidden_representation,target = torch.ones(inputs.size(0)).to(device))

            label_loss = ce_loss(studen_logits,labels)

            loss = hidden_rep_loss_weight * hidden_rep_loss + ce_loss_weight*label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loaders)}")
    



In [17]:
def test_outputs(model, test_loaders, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loaders:
            inputs , labels = inputs.to(device) , labels.to(device)

            outputs , _ = model(inputs)
            _ , predicted = torch.max(outputs.data,1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy : {accuracy:.2f}")
    return accuracy

In [20]:
train_cosine_loss(teacher=modified_nn_deep, student=modified_nn_light, train_loaders=train_loaders, epochs=50, learning_rate=5e-5, hidden_rep_loss_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_cosine_loss = test_outputs(modified_nn_light, test_loaders, device)

Epoch 1/50, Loss: 1.209270174241127
Epoch 2/50, Loss: 1.1756770205314813
Epoch 3/50, Loss: 1.1529140053197855
Epoch 4/50, Loss: 1.1325664690998205
Epoch 5/50, Loss: 1.115619655155465
Epoch 6/50, Loss: 1.1018539593957575
Epoch 7/50, Loss: 1.0868908183653947
Epoch 8/50, Loss: 1.0750122553552204
Epoch 9/50, Loss: 1.0646017195318667
Epoch 10/50, Loss: 1.0527007515778017
Epoch 11/50, Loss: 1.0446436037797757
Epoch 12/50, Loss: 1.0352031658677494
Epoch 13/50, Loss: 1.024368842239575
Epoch 14/50, Loss: 1.0166030632870278
Epoch 15/50, Loss: 1.0072967038130212
Epoch 16/50, Loss: 0.9997390784570933
Epoch 17/50, Loss: 0.9931838405711572
Epoch 18/50, Loss: 0.9857299847676017
Epoch 19/50, Loss: 0.9774245343854665
Epoch 20/50, Loss: 0.9697611142912179
Epoch 21/50, Loss: 0.9646281523777701
Epoch 22/50, Loss: 0.9577163709399036
Epoch 23/50, Loss: 0.9509349515675889
Epoch 24/50, Loss: 0.946033415587052
Epoch 25/50, Loss: 0.9402269868899489
Epoch 26/50, Loss: 0.9350691890472647
Epoch 27/50, Loss: 0.9282

In [21]:
class ModifiedDeepNNRegressor(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedDeepNNRegressor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        conv_feature_map = x
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, conv_feature_map

class ModifiedLightNNRegressor(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedLightNNRegressor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.regressor = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        regressor_output = self.regressor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, regressor_output

In [23]:
def train_mse_loss(teacher, student, train_loader, epochs, learning_rate, feature_map_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.to(device)
    student.to(device)
    teacher.eval() 
    student.train() 

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.no_grad():
                _, teacher_feature_map = teacher(inputs)

            student_logits, regressor_feature_map = student(inputs)

            hidden_rep_loss = mse_loss(regressor_feature_map, teacher_feature_map)

            label_loss = ce_loss(student_logits, labels)

            loss = feature_map_weight * hidden_rep_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

torch.manual_seed(42)
modified_nn_light_reg = ModifiedLightNNRegressor(num_classes=10).to(device)


modified_nn_deep_reg = ModifiedDeepNNRegressor(num_classes=10).to(device)
modified_nn_deep_reg.load_state_dict(nn_deep.state_dict())

<All keys matched successfully>

In [25]:
train_mse_loss(teacher=modified_nn_deep_reg, student=modified_nn_light_reg, train_loader=train_loaders, epochs=50, learning_rate=5e-5, feature_map_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_mse_loss = test_outputs(modified_nn_light_reg, test_loaders, device)

Epoch 1/50, Loss: 33.94429340996706
Epoch 2/50, Loss: 24.149619846392774
Epoch 3/50, Loss: 22.244878217692264
Epoch 4/50, Loss: 20.668229783587442
Epoch 5/50, Loss: 19.198318949745744
Epoch 6/50, Loss: 17.67490135495315
Epoch 7/50, Loss: 16.07704192842059
Epoch 8/50, Loss: 14.544416320293456
Epoch 9/50, Loss: 13.29257458250236
Epoch 10/50, Loss: 12.269069025278702
Epoch 11/50, Loss: 11.3274664305665
Epoch 12/50, Loss: 10.373412415194695
Epoch 13/50, Loss: 9.378615784218244
Epoch 14/50, Loss: 8.437383891981277
Epoch 15/50, Loss: 7.693902702282761
Epoch 16/50, Loss: 7.166577001361896
Epoch 17/50, Loss: 6.756850480423559
Epoch 18/50, Loss: 6.416731303915038
Epoch 19/50, Loss: 6.129864102434319
Epoch 20/50, Loss: 5.885206888398856
Epoch 21/50, Loss: 5.669684583268812
Epoch 22/50, Loss: 5.487340044182584
Epoch 23/50, Loss: 5.317224620858116
Epoch 24/50, Loss: 5.171969450343296
Epoch 25/50, Loss: 5.040686625653826
Epoch 26/50, Loss: 4.923521397973571
Epoch 27/50, Loss: 4.813330021050885
Epoc

In [26]:
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + CosineLoss: {test_accuracy_light_ce_and_cosine_loss:.2f}%")
print(f"Student accuracy with CE + RegressorMSE: {test_accuracy_light_ce_and_mse_loss:.2f}%")

Teacher accuracy: 71.77%
Student accuracy without teacher: 66.32%
Student accuracy with CE + CosineLoss: 65.00%
Student accuracy with CE + RegressorMSE: 65.98%


In [29]:
torch.save(modified_nn_light.state_dict(), "ModefiedNNLight.pth")
torch.save(modified_nn_light_reg.state_dict(),"ModifiedNNLightReg.pth")
print("Final model saved!")

Final model saved!


In [23]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import os

class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                'dog', 'frog', 'horse', 'ship', 'truck']

transform = transforms.Compose([
    transforms.Resize((32, 32)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])  
])

def load_model(model_path : str):
    model = DeepNN()
    model.load_state_dict(torch.load(model_path,map_location=device,weights_only=False))
    model.to(device)
    model.eval()
    return model


def predict_image(image_path, model_path, device=device):
    model = load_model(model_path)

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  

    with torch.no_grad():
        output = model(image)
    
    probabilities = F.softmax(output, dim=1)
  
    predicted_class_idx = torch.argmax(probabilities, dim=1).item()
    predicted_class = class_labels[predicted_class_idx]

    return predicted_class, probabilities.cpu().numpy()

result = []
model_path = "./NNDeep.pth"  
folder_path = "./EvalDataset"

for filename in os.listdir(folder_path):
    if filename.endswith((".png",".jpg",".jpeg")):
        image_path = os.path.join(folder_path,filename)
        predicted_label, probabilities = predict_image(image_path, model_path)
        result.append({
            "image" : filename,
            "predicted_class":predicted_label,
            "probabilities" : probabilities.tolist()
        })
print(result)

[{'image': 'Cars.png', 'predicted_class': 'bird', 'probabilities': [[0.0029290253296494484, 1.4356356814460014e-07, 0.977946937084198, 0.0004215117951389402, 0.01677001267671585, 0.0001279014250030741, 4.4039975932719244e-07, 0.0017982993740588427, 1.0911135994717824e-08, 5.6159269661293365e-06]]}, {'image': 'CyberTruck.jpg', 'predicted_class': 'bird', 'probabilities': [[0.005296729505062103, 0.07268933206796646, 0.6597440242767334, 9.25747663131915e-05, 0.0001061474031303078, 2.6200650609098375e-06, 0.00024029589258134365, 0.00453138817101717, 5.154713562660618e-07, 0.25729644298553467]]}, {'image': 'Deer.jpg', 'predicted_class': 'deer', 'probabilities': [[1.4864350639243185e-08, 1.0849383613731334e-08, 0.006579791195690632, 2.697348827496171e-05, 0.9917028546333313, 0.0015323807019740343, 1.2202338695033177e-08, 0.0001578085357323289, 4.145479803614044e-13, 2.0534912437142339e-07]]}, {'image': 'Dogimage.jpg', 'predicted_class': 'dog', 'probabilities': [[5.954097631599709e-11, 1.81879

In [25]:
import onnx

dummy_input = torch.randn(1,3,32,32).to(device)
onnx_path = "ImagePredictor.onnx"
model = load_model(model_path="./NNDeep.pth")

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input" : {0 : "batch_size"},
        "output" : {0 : "batch_size"}
    }
)

print(f"Model converted to ONNX: {onnx_path}")

Model converted to ONNX: ImagePredictor.onnx


In [1]:
from onnxruntime.quantization import quantize_dynamic , QuantType

quantized_onnx_path = "ImagePredictorQuantizedFinalized.onnx"

onnx_path = "ImagePredictor.onnx"

quantized_model = quantize_dynamic(
    onnx_path,
    quantized_onnx_path,
    weight_type=QuantType.QUInt8
)
print(f"Quantized ONNX Model: {quantized_onnx_path}")



Quantized ONNX Model: ImagePredictorQuantizedFinalized.onnx
